# Train Spacenet from snapshot on a single case

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [1]:
import cv2
import numpy as np
from PIL import Image
import sys

In [2]:
sys.path.append('/home/catskills/Desktop/xview2-catskills/utils')

In [3]:
import chainer
import chainer.functions as F
from chainer import cuda, serializers, Variable
from inference import inference
output='foo.json'

## First do inference inside notebook to see how to apply the net

In [4]:
import cv2
import numpy as np
from PIL import Image
import sys
import chainer
import chainer.functions as F
from chainer import cuda, serializers, Variable
from segmentation import SegmentationModel as Model

input='/home/catskills/Desktop/dataxv2/xBD/santa-rosa-wildfire/images/santa-rosa-wildfire_00000030_pre_disaster.png'
trainfn='/home/catskills/Desktop/dataxv2/xBD/santa-rosa-wildfire/masks/santa-rosa-wildfire_00000030_pre_disaster.png'
weights='/home/catskills/Desktop/dataxv2/release/v_catskills_0.2.1/localization.hdf5'
mean='/home/catskills/Desktop/xview2-catskills/weights/mean.npy'
sys.path.append('../src/models')
mean = np.load(mean)
model = Model(weights, mean)
image = np.array(Image.open(input))
train=np.array(Image.open(trainfn))

In [5]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [6]:
train = rgb2gray(np.array(Image.open(trainfn)))>0.5

In [9]:
score = model.apply_segmentation(image)
building_mask_pred = (np.argmax(score, axis=0) == 1)
bm_only = (building_mask_pred & ~train).astype(int)*9
train_only=(train & ~building_mask_pred).astype(int)*3
bm_and_tr=(building_mask_pred & train).astype(int)*6
all = bm_only + train_only + bm_and_tr

## Overtrain Santa Rosa 30

In [10]:
def float_to_cpu(x):
    return float(cuda.to_cpu(x.data))

optimizer = chainer.optimizers.Adam()
optimizer.setup(model._SegmentationModel__model);

train_int = train.astype(int)
gt=np.array([train_int])
gt_in = Variable(cuda.cupy.asarray(gt, dtype=cuda.cupy.int))

In [11]:
image_in, crop = model._SegmentationModel__preprocess(image)

from imantics import Mask

from simplification.cutil import simplify_coords_vwp

def predict_polygons(polygons):
    new_predictions = []
    for poly in polygons:
        if len(poly) >= 3:
            f = poly.reshape(-1, 2)
            simplified_vw = simplify_coords_vwp(f, .3)
            if len(simplified_vw) > 2:
                    mpoly = []
                    # Rebuilding the polygon in the way that PIL expects the values [(x1,y1),(x2,y2)]
                    for i in simplified_vw:
                        mpoly.append((i[0], i[1]))
                    # Adding the first point to the last to close the polygon
                    mpoly.append((simplified_vw[0][0], simplified_vw[0][1]))
                    new_predictions.append(mpoly)
    return new_predictions

def polygon_loss(score_cuda, n_gt_polygons):
    score1 = F.softmax(score_cuda)
    score1_cpu = cuda.to_cpu(score1.data)[0]
    building_mask_pred = (np.argmax(score1_cpu, axis=0) == 1)
    polygons_pred = Mask(building_mask_pred).polygons()
    n_polygons_pred = len(predict_polygons(polygons_pred))
    if n_gt_polygons == 0 and n_polygons_pred > 0:
        poly_loss = 1
    else: 
        poly_loss = abs(n_polygons_pred-n_gt_polygons)/n_gt_polygons
    return poly_loss, n_gt_polygons, n_polygons_pred

gt_polygons = Mask(train).polygons()

n_gt_polygons = len(predict_polygons(gt_polygons))

for i in range(10):
    with chainer.using_config('train', True):
        score_cuda = model._SegmentationModel__model.forward(image_in)
        loss = F.softmax_cross_entropy(score_cuda, gt_in)    
        poly_loss, n_gt, n_pred = polygon_loss(score_cuda, n_gt_polygons)
        softmax_loss = float_to_cpu(loss)
        loss += 0.4*poly_loss
        accuracy=float_to_cpu(F.accuracy(score_cuda, gt_in))
        print(f'[{i}] n_gt {n_gt} n_pred {n_pred} poly_loss {100*poly_loss:.1f}% softmax_loss {100*softmax_loss:.1f}% loss {100*float_to_cpu(loss):.3f}% accuracy {100*accuracy:.6f}%')
        model._SegmentationModel__model.cleargrads()
        loss.backward()
        optimizer.update()

[0] n_gt 34 n_pred 65 poly_loss 91.2% softmax_loss 6.4% loss 42.882% accuracy 97.634220%
[1] n_gt 34 n_pred 67 poly_loss 97.1% softmax_loss 5.3% loss 44.145% accuracy 97.947884%
[2] n_gt 34 n_pred 61 poly_loss 79.4% softmax_loss 4.7% loss 36.464% accuracy 98.223114%
[3] n_gt 34 n_pred 51 poly_loss 50.0% softmax_loss 4.3% loss 24.267% accuracy 98.407841%
[4] n_gt 34 n_pred 48 poly_loss 41.2% softmax_loss 4.0% loss 20.422% accuracy 98.499393%
[5] n_gt 34 n_pred 39 poly_loss 14.7% softmax_loss 3.7% loss 9.594% accuracy 98.561192%
[6] n_gt 34 n_pred 44 poly_loss 29.4% softmax_loss 3.5% loss 15.282% accuracy 98.608589%
[7] n_gt 34 n_pred 42 poly_loss 23.5% softmax_loss 3.3% loss 12.760% accuracy 98.654652%
[8] n_gt 34 n_pred 41 poly_loss 20.6% softmax_loss 3.2% loss 11.432% accuracy 98.705292%
[9] n_gt 34 n_pred 40 poly_loss 17.6% softmax_loss 3.1% loss 10.124% accuracy 98.756409%


In [ ]:
with chainer.using_config('train', True):
    score_cuda= model._SegmentationModel__model.forward(image_in)
score_cuda_softmax = F.softmax(score_cuda)
score_cpu = cuda.to_cpu(score_cuda_softmax.data)[0]
top, left, bottom, right = crop
score_cpu = score_cpu[:, top:bottom, left:right]
building_mask_pred = (np.argmax(score_cpu, axis=0) == 1)

bm_only = (building_mask_pred & ~train).astype(int)*9
train_only=train & ~building_mask_pred.astype(int)*3
bm_and_tr=(building_mask_pred & train).astype(int)*6
all = bm_only + train_only + bm_and_tr
plt.figure(figsize=(10,10));
plt.imshow(all)

In [ ]:
plt.imshow(building_mask_pred)

In [ ]:
plt.imshow(building_mask_pred & ~train);

In [ ]:
plt.imshow(~building_mask_pred & train);

In [ ]:
plt.imshow(building_mask_pred ^ train);

In [ ]:
from chainer import serializers
new_weights='model_updated.hdf5'
serializers.save_npz(new_weights, model._SegmentationModel__model)

import chainer.computational_graph as c
g = c.build_computational_graph(score_cuda)
with open('loss.dot', 'w') as o:
    o.write(g.dump())